In [ ]:
#Get Neo4J official Python driver
!pip install neo4j

In [ ]:
def connect(uri, username, password):
    try:
        return GraphDatabase.driver(uri, auth=(username, password))
    except ClientError as e:
        print(e)
        return -1

In [ ]:
def create_and_return_greeting(tx, message):
    result = tx.run("CREATE (a:Greeting) "
                    "SET a.message = $message "
                    "RETURN a.message + ', from node ' + id(a)", message=message)
    return result.single()[0]

In [ ]:
def add_user(tx, username, password):
    try:
        result = tx.run("CALL dbms.security.createUser($username,$password, false)", username=username, password=password)
    except ClientError as e:
        print(e)

In [ ]:
def change_password(tx, password):
    try:
        result = tx.run("CALL dbms.security.changePassword($password)", password=password)
    except ClientError as e:
        print(e)

In [ ]:
#Neo4J connection test
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError

uri = "bolt://localhost:7687"
our_username = "ads"
our_password = "ads"
message = "Response"
default = "neo4j"

# Logging with default user and password
driver = connect(uri, default, default)
# If we have already run this before, then the password is already changed
if driver != -1:
    ''' But if it is the first time we log as that default user, 
        we are forced to change the password '''
    with driver.session() as session:
        session.write_transaction(change_password, our_password) # e.g. our_password
    # We should exit and connect again after changing the password
    driver.close()

driver = connect(uri, default, our_password)

# Add the user that will be used for the rest of operations
with driver.session() as session:
    try: 
        session.write_transaction(add_user, our_username, our_password)
    except ClientError as e:
        print(e)
driver.close()

driver = connect(uri, our_username, our_password)
with driver.session() as session:
    greeting = session.write_transaction(create_and_return_greeting, message)
    print(greeting) 
driver.close()